In [64]:
import pandas as pd
import numpy as np

In [65]:
dtypes = {
    'country_name': str,
    'expenses': object,
    'id': str,
    'title': str,
    'state_name': str,
    'lat': float,
    'lon': float
}
cols = range(6) + range(8, 18) + [20, 21] + range(23, 27)
df = pd.read_csv('Desktop/Homework/data_gba_total.csv', dtype=dtypes, usecols=cols)

In [66]:
def split_region(x):
    state = x.split('|')[2]
    return state

def split_neighborhood(x):
    state = x.split('|')[3]
    return state

df['state'] = df.place_with_parent_names.apply(split_region)
df['barrio'] = df.place_with_parent_names.apply(split_neighborhood)

In [67]:
df = df.loc[(df['created_on'] >= '2016-08-08'),:]

In [68]:
list_estados = []

def state(row):
    if pd.isnull(row['state']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['state'] in list_estados:
        return list_estados.index(row['state'])+1
    
    list_estados.append(row['state'])
    
    return list_estados.index(row['state'])+1

df['state_n'] = df.apply(state, axis=1)

In [69]:
list_barrio = []

def barrio(row):
    if pd.isnull(row['barrio']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['barrio'] in list_barrio:
        return list_barrio.index(row['barrio'])+1
    
    list_barrio.append(row['barrio'])
    
    return list_barrio.index(row['barrio'])+1

df['barrio_n'] = df.apply(barrio, axis=1)

In [70]:
list_property = []

def property_type(row):
    if pd.isnull(row['property_type']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['property_type'] in list_property:
        return list_property.index(row['property_type'])+1
    
    list_property.append(row['property_type'])
    
    return list_property.index(row['property_type'])+1

df['property_type_n'] = df.apply(property_type, axis=1)

In [71]:
def extra(row, list_of_words):
    if pd.isnull(row['title']):
        return 0
    for word in list_of_words:
        if word in row['title']:
            return 1
    return 0

df['pileta'] = df.apply((lambda x: extra(x, ['pile'])), axis=1)
df['jardin'] = df.apply((lambda x: extra(x, ['patio', 'jardin'])), axis=1)
df['garage'] = df.apply((lambda x: extra(x, ['garage', 'cochera'])), axis=1)

# Tomo solamente los datos que me interesan

In [72]:
df = df.loc[:, ['lat', 'lon', 'price_aprox_usd', 'rooms', 'surface_covered_in_m2', 'state_n', 'barrio_n', 'property_type_n'
               , 'pileta', 'jardin', 'garage']]

In [73]:
df = df.dropna(subset=['price_aprox_usd'], how='any')

In [74]:
df.shape

(428696, 11)

# Creo un set de pruebas como el dado por kaggle

In [75]:
df_test = df.loc[(np.isnan(df['surface_covered_in_m2']))|(np.isnan(df['lat']))|(np.isnan(df['lon']))|(np.isnan(df['rooms']))]

In [76]:
df_test.shape

(228867, 11)

In [14]:
df.to_csv('Desktop/Homework/dataset.csv')
df_test.to_csv('Desktop/Homework/dataset_pruebas.csv')

# Creo dataset para predecir

In [34]:
properati = pd.read_csv('Desktop/Homework/properati_dataset_testing_noprice.csv')

In [35]:
properati.index = properati['id']

In [36]:
properati.head()

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description
id,,,,,,,,,,,,,,,,,
3632,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.0,NaN,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...
3633,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...
2263404,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.0,48.0,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...
2263405,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
2263406,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,51.0,51.0,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...


In [37]:
properati['state'] = properati.place_with_parent_names.apply(split_region)
properati['barrio'] = properati.place_with_parent_names.apply(split_neighborhood)

# Paso a ingles los tipo de propiedad

In [38]:
properati.groupby('property_type').count()

,id,created_on,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,state,barrio
property_type,,,,,,,,,,,,,,,,,,
casa,3409,3409,3409,3409,3409,3409,3409,2401,2401,2401,2535,3029,55,1731,75,3409,3409,3409
departamento,9761,9761,9761,9761,9761,9761,9761,7330,7330,7330,8507,9079,1242,5167,2407,9761,9761,9761
ph,996,996,996,996,996,996,996,756,756,756,811,897,71,602,61,996,996,996


In [39]:
df_para_ver_tipos = pd.read_csv('Desktop/Homework/data_gba_total.csv', dtype=dtypes, usecols=cols)

In [40]:
df_para_ver_tipos.groupby('property_type').count()

,country_name,created_on,currency,expenses,floor,geonames_id,lat,lat-lon,lon,operation,...,place_with_parent_names,price,price_aprox_local_currency,price_aprox_usd,price_per_m2,rooms,surface_covered_in_m2,surface_in_m2,surface_total_in_m2,title
property_type,,,,,,,,,,,,,,,,,,,,,
PH,48669,144101,130846,3874,7542,125466,124634,124634,124634,144101,...,144101,132168,132168,132168,42286,91076,46619,61109,38707,86727
apartment,431839,1379444,1220754,114012,394859,1169563,1132140,1132140,1132129,1379444,...,1379444,1228704,1228702,1228702,385720,965352,425289,656427,337330,816304
house,274241,833897,694308,12033,18258,619257,549195,549195,549195,833897,...,833897,701426,701426,701426,224084,357266,258852,397424,234632,543895
store,25436,52029,42793,1727,761,44419,40728,40728,40728,52029,...,52029,43391,43391,43391,19154,4668,23294,15310,19794,44216


In [41]:
def pasar_a_ingles(row):
    if row['property_type']=='casa':
        return 'house'
    if row['property_type']=='departamento':
        return 'apartment'
    if row['property_type']=='ph':
        return 'PH' 
    
    return 0

properati['property_type'] = properati.apply(pasar_a_ingles, axis=1)

In [42]:
properati.groupby('property_type').count()

,id,created_on,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,state,barrio
property_type,,,,,,,,,,,,,,,,,,
PH,996,996,996,996,996,996,996,756,756,756,811,897,71,602,61,996,996,996
apartment,9761,9761,9761,9761,9761,9761,9761,7330,7330,7330,8507,9079,1242,5167,2407,9761,9761,9761
house,3409,3409,3409,3409,3409,3409,3409,2401,2401,2401,2535,3029,55,1731,75,3409,3409,3409


In [43]:
properati['state_n'] = properati.apply(state, axis=1)
properati['barrio_n'] = properati.apply(barrio, axis=1)
properati['property_type_n'] = properati.apply(property_type, axis=1)

In [44]:
properati['title']=properati['description']
properati['pileta'] = properati.apply((lambda x: extra(x, ['pile'])), axis=1)
properati['jardin'] = properati.apply((lambda x: extra(x, ['patio', 'jardin'])), axis=1)
properati['garage'] = properati.apply((lambda x: extra(x, ['garage', 'cochera'])), axis=1)

# Tomo solo las columnas que me interesan

In [45]:
properati = properati.loc[:, ['lat', 'lon', 'rooms', 'surface_covered_in_m2', 'state_n', 'barrio_n', 'property_type_n'
               , 'pileta', 'jardin', 'garage']]

In [46]:
properati.head()

,lat,lon,rooms,surface_covered_in_m2,state_n,barrio_n,property_type_n,pileta,jardin,garage
id,,,,,,,,,,
3632,-34.610988,-58.363464,NaN,NaN,1,22,1,0,0,1
3633,NaN,NaN,NaN,NaN,5,4,1,1,0,0
2263404,-34.589363,-58.412880,NaN,48.0,1,12,1,0,0,0
2263405,NaN,NaN,NaN,39.0,1,45,1,0,0,0
2263406,NaN,NaN,NaN,51.0,1,45,1,0,0,0


In [47]:
properati.to_csv('Desktop/Homework/properati.csv')